# 1. Imports

In [1]:
import os
import numpy as np
import pandas as pd
from pandas.core.common import flatten
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
import seaborn as sns
import platform
from tqdm import tqdm
import random
import glob
import copy
from dataclasses import dataclass
import time
from collections import Counter
from collections import defaultdict
import gc

import sklearn
from sklearn.metrics import classification_report, f1_score, precision_score, recall_score, confusion_matrix, roc_curve, auc, roc_auc_score

# import albumentations as A
# from albumentations.pytorch import ToTensorV2

# import torch
# from torch.utils.data import Dataset, DataLoader
# import torch.nn as nn
# import torchvision
# import torchvision.models as models
# import torch.nn.functional as nnf

# import timm
import cv2
import gc
import tensorflow as tf
from tensorflow.keras.utils import to_categorical

import warnings
warnings.filterwarnings("ignore")


2024-03-29 09:05:42.082289: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-29 09:05:42.457375: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-29 09:05:43.199378: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
@dataclass
class CONFIG:
    TRAIN_PATH = "../00_data/dataset/augmented"
    VAL_PATH = "../00_data/dataset/val"
    MODEL_NAMES = ['densenet161', 'mobilevitv2_150', 'mobilevitv2_150_384_in22ft1k']
    BATCH_SIZE = 8
    LEARNING_RATE = 1e-4
    DROPOUT = 0.3
    EPOCHS = 10

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    
# def get_device():
#     global DEVICE
#     if torch.cuda.is_available():
#         print("[INFO] Using GPU: {}\n".format(torch.cuda.get_device_name()))
#         DEVICE = torch.device("cuda:0")
#     else:
#         print("\n[INFO] GPU not found. Using CPU: {}\n".format(platform.processor()))
#         DEVICE = torch.device("cpu")
    
RANDOM_SEED = 42
set_seed(RANDOM_SEED)
# get_device()

# 2. Importing Data

In [3]:
no = np.load("../../00_data/original_data/no.npy")
sphere = np.load("../../00_data/original_data/sphere.npy")
vortex = np.load("../../00_data/original_data/vortex.npy")

In [4]:
y_no = np.zeros(len(no))
y_sphere = np.ones(len(sphere))
y_vortex = np.full(len(vortex), 2)

In [5]:

y_no = to_categorical(y_no, num_classes=3)
y_sphere = to_categorical(y_sphere, num_classes=3)
y_vort = to_categorical(y_vortex, num_classes=3)

In [6]:
x_data = np.concatenate([no, sphere, vortex], axis=0)
y_data = np.concatenate([y_no, y_sphere, y_vort], axis=0)


In [7]:
y_data.shape

(30000, 3)

In [8]:

del(no,sphere,vortex)

gc.collect()


33

import tensorflow as tf

def convert_grayscale_images_to_rgb(images):
    """
    Convert a batch of grayscale images to RGB by replicating the grayscale
    channel three times.

    Parameters:
    - images: A 3D NumPy array or TensorFlow tensor of shape [samples, height, width].

    Returns:
    - A 4D TensorFlow tensor of shape [samples, height, width, 3].
    """
    # Add a channel dimension, making it [samples, height, width, 1]

    # Replicate the channel 3 times, resulting in [samples, height, width, 3]
    images_rgb = tf.tile(images, [1, 1, 1, 3])
    return images_rgb

# Example usage with your data:
x_data = convert_grayscale_images_to_rgb(x_data)


In [9]:
x_data.shape

(30000, 150, 150, 1)

# 2.1 Prepare Dataset 

In [10]:
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(x_data, y_data, test_size=0.2, random_state=RANDOM_SEED, stratify=y_data)


In [11]:
def prepare_data(x_train, y_train, x_test, y_test, batch_size):
    train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(len(x_train)).batch(batch_size)
    test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(batch_size)
    return train_dataset, test_dataset

# 3. Model

In [14]:
import tensorflow as tf

class MobileVitV2_150(tf.keras.Model):
    
    def __init__(self, n_classes):
        super(MobileVitV2_150, self).__init__()
        
        # Assuming an equivalent model is loaded here.
        # TensorFlow Hub could be a source for such models.
        self.vit_model = tf.keras.applications.MobileNetV2(input_shape=(150, 150, 1), include_top=False, weights='imagenet')
        self.vit_model.trainable = True  # Make the model trainable
        
        self.classifier = tf.keras.Sequential([
            tf.keras.layers.GlobalAveragePooling2D(),
            tf.keras.layers.Dense(512, activation='relu'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Dropout(0.3),  # Assuming using the dropout from CONFIG
            tf.keras.layers.Dense(64, activation='relu'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Dropout(0.3),  # Assuming using the dropout from CONFIG
            tf.keras.layers.Dense(n_classes, activation='softmax')
        ])
        
    def call(self, inputs):
        x = self.vit_model(inputs, training=True)
        x = self.classifier(x)
        return x

class TransferLearningModelNew(tf.keras.Model):
    def __init__(self, n_classes):
        super(TransferLearningModelNew, self).__init__()
        
        # Load a pre-trained model. Assuming DenseNet161 equivalent in TensorFlow.
        # Note: TensorFlow models by default use 3 input channels. If you need a single-channel model,
        # you may need to adjust the input layer or preprocess your input data accordingly.
        self.base_model = tf.keras.applications.DenseNet169(include_top=False, input_shape=(150, 150, 1), weights=None)
        self.base_model.trainable = True  # Enable training on the base model
        
        # Assuming the feature extraction from the base model results in a specific shape, adjust accordingly.
        # The number of features (2208 * 4 * 4) needs to be updated based on the output shape of your specific base model.
        self.global_pool = tf.keras.layers.GlobalAveragePooling2D()
        self.classifier = tf.keras.Sequential([
            tf.keras.layers.Dense(1024, activation='relu'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Dropout(0.33),
            tf.keras.layers.Dense(64, activation='relu'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Dropout(0.33),
            tf.keras.layers.Dense(n_classes, activation='softmax')  # Use 'softmax' for multi-class classification
        ])
        
    def call(self, inputs, training=False):
        x = self.base_model(inputs, training=training)
        x = self.global_pool(x)
        x = self.classifier(x, training=training)
        return x
    

class DenseNet201(tf.keras.Model):
    def __init__(self, n_classes):
        super(DenseNet201, self).__init__()
        # Initialize the DenseNet201 base model
        self.base_model = tf.keras.applications.DenseNet201(include_top=False,
                                                            input_shape=(150, 150, 1),
                                                            weights='imagenet')
        self.base_model.trainable = True  # Set True to fine-tune all layers

        # Define the custom classifier
        self.flatten = tf.keras.layers.Flatten()
        self.classifier = tf.keras.Sequential([
            tf.keras.layers.Dense(1024, activation='relu'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Dropout(0.33),
            tf.keras.layers.Dense(64, activation='relu'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Dropout(0.33),
            tf.keras.layers.Dense(n_classes, activation='softmax')  # Use 'sigmoid' for binary classification
        ])
        
    def call(self, inputs, training=False):
        x = self.base_model(inputs, training=training)
        x = self.flatten(x)
        x = self.classifier(x, training=training)
        return x

class DenseNetEnsemble(tf.keras.Model):
    def __init__(self, modela, modelb):
        super(DenseNetEnsemble, self).__init__()
        self.modela = modela
        self.modelb = modelb
        
    def call(self, inputs, training=False):
        outa = self.modela(inputs, training=training)
        outb = self.modelb(inputs, training=training)
        # Assuming the models are for classification and use softmax, averaging predictions is a common approach.
        # If you're directly adding outputs (e.g., for regression), you might simply use: out = outa + outb
        out = (outa + outb) / 2.0
        return out


In [15]:
n_classes = 3  # Ensure 'classes' is defined

# Initialize the component models
# modela_tf = TransferLearningModelNew(n_classes=n_classes)
modelb_tf = DenseNet201(n_classes=n_classes)

# Create the ensemble model
# ensemble_model_tf = DenseNetEnsemble(modela=modela_tf, modelb=modelb_tf)
model = modelb_tf


2024-03-29 09:06:50.232641: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-29 09:06:50.385936: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-29 09:06:50.385973: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-29 09:06:50.388827: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-29 09:06:50.388864: I external/local_xla/xla/stream_executor

In [ ]:
def train_one_epoch(model, optimizer, train_dataset, loss_fn):
    train_loss = []
    train_accuracy = []

    for images, labels in train_dataset:
        with tf.GradientTape() as tape:
            predictions = model(images, training=True)
            loss = loss_fn(labels, predictions)
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))
        
        train_loss.append(loss)
        correct = tf.reduce_sum(tf.cast(tf.equal(tf.argmax(predictions, axis=1), labels), tf.float32))
        train_accuracy.append(correct / len(labels))
    
    return tf.reduce_mean(train_accuracy).numpy(), tf.reduce_mean(train_loss).numpy()

def test_model(model, test_dataset, loss_fn):
    test_loss = []
    test_accuracy = []

    for images, labels in test_dataset:
        predictions = model(images, training=False)
        loss = loss_fn(labels, predictions)
        
        test_loss.append(loss)
        correct = tf.reduce_sum(tf.cast(tf.equal(tf.argmax(predictions, axis=1), labels), tf.float32))
        test_accuracy.append(correct / len(labels))
    
    return tf.reduce_mean(test_accuracy).numpy(), tf.reduce_mean(test_loss).numpy()


In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=CONFIG.LEARNING_RATE)

loss_function = tf.keras.losses.CategoricalCrossentropy(from_logits=True)

In [ ]:
gc.collect()

0

In [ ]:
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# Assuming CONFIG is a configuration object similar to the PyTorch setup
# and you have x_train, y_train, x_val, y_val prepared

model_name = "ensemble"
best_model_path = f'{model_name}_epochs_{CONFIG.EPOCHS}_batchsize_{CONFIG.BATCH_SIZE}_lr_{CONFIG.LEARNING_RATE}.keras'

# Prepare your datasets (assuming x_train, y_train, x_val, y_val are your datasets)
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(buffer_size=10000).batch(CONFIG.BATCH_SIZE)
val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val)).batch(CONFIG.BATCH_SIZE)

# Define the model, loss function, and optimizer
 # Initialize your TensorFlow model here

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=CONFIG.LEARNING_RATE),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

# Define callbacks
callbacks = [
    ModelCheckpoint(best_model_path, save_best_only=True, monitor='val_accuracy', mode='max', verbose=1),
    EarlyStopping(monitor='val_accuracy', patience=10, verbose=1, mode='max', restore_best_weights=True)
]




In [ ]:
x_val.shape

(6000, 150, 150, 1)

In [ ]:
y_val.shape

(6000, 3)

In [ ]:
# Train the model
history = model.fit(train_dataset, validation_data=val_dataset, epochs=CONFIG.EPOCHS, callbacks=callbacks)

# Optionally, if you want to directly access the best accuracy achieved:
best_accuracy = max(history.history['val_accuracy'])
print(f"Best validation accuracy: {best_accuracy}")

Epoch 1/10


I0000 00:00:1711698319.041356    3789 service.cc:145] XLA service 0x7f49040bb0d0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1711698319.041415    3789 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce RTX 4080, Compute Capability 8.9
2024-03-29 07:45:21.974634: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-03-29 07:45:30.938902: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8907
I0000 00:00:1711698347.580613    4067 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_100123', 36 bytes spill stores, 36 bytes spill loads

I0000 00:00:1711698513.131527    3789 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


3000/3000 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - accuracy: 0.3352 - loss: 1.1201
Epoch 1: val_accuracy improved from -inf to 0.32617, saving model to ensemble_epochs_10_batchsize_8_lr_0.0001.keras
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 543s 88ms/step - accuracy: 0.3352 - loss: 1.1201 - val_accuracy: 0.3262 - val_loss: 1.1125
Epoch 2/10
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - accuracy: 0.3389 - loss: 1.1136
Epoch 2: val_accuracy improved from 0.32617 to 0.33200, saving model to ensemble_epochs_10_batchsize_8_lr_0.0001.keras
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 244s 81ms/step - accuracy: 0.3389 - loss: 1.1136 - val_accuracy: 0.3320 - val_loss: 1.1129
Epoch 3/10
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - accuracy: 0.3337 - loss: 1.1131
Epoch 3: val_accuracy did not improve from 0.33200
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 239s 79ms/step - accuracy: 0.3337 - loss: 1.1131 - val_accuracy: 0.3245 - val_loss: 1.1068
Epoch 4/10
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - accuracy: 0.3357 - loss: 1.1128
Epo